In [ ]:
import datetime
import json
import os
import pandas as pd
import requests

In [ ]:
class NapsterAPIcall():

    API_URL = 'https://api.napster.com/'
    API_VERSION = 'v2.2'
    API_KEY = os.environ['napster_API_KEY']
    API_SECRET = os.environ['napster_API_SECRET']
    
    user = os.environ['napster_user']
    password = os.environ['napster_password']
    
    headers = None
    
    def __init__(self):
        print('init')
    
    def login(self):
        # Login
        OAUTH_URL =  self.API_URL + 'oauth/token'
        params = (self.API_KEY, self.API_SECRET)
        data = {
            'username': self.user,
            'password': self.password,
            'grant_type': 'password'
            }
        response_login = requests.post(url = OAUTH_URL, auth = params, data = data)
        tokens = json.loads(response_login.text)
        self.headers = {'Authorization': 'Bearer ' + tokens['access_token']}
        
        return response_login.status_code
    

    def fetchAccountInfo(self):
    # Account Information
        response = requests.get(url = self.API_URL + self.API_VERSION + '/me/account', headers = self.headers)
        return(response)

    def _fetchFavoriteTracks(self, offset):
    # Favorite Tracks
        params = {'limit': 200, 'offset': offset, 'filter': 'track'}
        response = requests.get(url = self.API_URL + self.API_VERSION + '/me/favorites', headers = self.headers, params = params)
        #file = open("favorites.json","w+")
        #file.write(response.text)
        #file.close()
        return response.status_code, json.loads(response.text)
    
    def fetchFavorites(self):
    # All Favorite Tracks
        columns = ['artist', 'title', 'streamable']
        favorites_df = pd.DataFrame(columns = columns)
        ind = 0
        while(True):
            code, favorites = self._fetchFavoriteTracks(offset = ind*200)
            if code == 200:
                #print(favorites_tracks)
                favorites_tracks = favorites['favorites']['data']['tracks']
                for item in favorites_tracks:
                    title, artist, streamable = self.extractFromList(item)
                    favorites_df = favorites_df.append(pd.DataFrame(data = [[artist, title, streamable]], 
                                                            columns = columns), ignore_index = True)
                # test for end of list
                lim = favorites['meta']['query']['limit']
                off = favorites['meta']['query']['offset']
                if favorites['meta']['totalCount'] <= (off * 200 + lim):
                    break
                
                ind += 1
            else:
                print('something went wrong!')
                return
        
        return favorites_df
    
    def extractFromList(self, item):
        title = item['name']
        artist = item['artistName']
        streamable = item['isStreamable']
        return title, artist, streamable
    
    def fetchfromLibrary(self, item, subitem = None):
        itemstr = ''
        if subitem is None:
            itemstr = item
        else:
            itemstr = item + '/' + subitem
        params = {'limit': 200, 'offset': 0}
        response = requests.get(url = self.API_URL + self.API_VERSION + '/me/library/' + itemstr, 
                                headers = self.headers, params = params)
        return response.status_code, json.loads(response.text)
    
    def fetchPlaylist(self, playlist):
        tracks_list = []
        code, response = apicall.fetchfromLibrary(item = 'playlists', subitem = playlist + '/tracks')
        for item in response['tracks']:
            title, artist, streamable = apicall.extractFromList(item)
            tracks_list.append([title, artist, streamable])
        
        tracks_df = pd.DataFrame(tracks_list, columns=['title', 'artist', 'streamable'])
        return tracks_df
        
        

In [ ]:
def downloadCurrentFavorites():
    favorites_df = apicall.fetchFavorites()
    now = datetime.datetime.now().strftime('%Y-%m-%d')
    favorites_df.to_csv('favorites_' + now + '.csv', index = 0)

def checkAllPlaylists():
    code, response = apicall.fetchfromLibrary(item = 'playlists')
    listPlaylist = [[item['id'], item['name']] for item in response['playlists']]
    columns = ['playlist','artist', 'title', 'streamable']
    unstream_tracks_df = pd.DataFrame(columns = columns)
    for listitem in listPlaylist:
        code, response = apicall.fetchfromLibrary(item = 'playlists', subitem = listitem[0] + '/tracks')
        for item in response['tracks']:
            title, artist, streamable = apicall.extractFromList(item)
            if not streamable:
                unstream_tracks_df = unstream_tracks_df.append(pd.DataFrame(data = [[listitem[1], artist, title, streamable]], 
                                                                            columns = columns), ignore_index = True)
        #print(response)
    now = datetime.datetime.now().strftime('%Y-%m-%d')
    unstream_tracks_df.to_csv('unstreamTracks_playlists_' + now + '.csv', index = 0)
    return unstream_tracks_df

In [ ]:
apicall = NapsterAPIcall()
apicall.login()

#resp = apicall.fetchAccountInfo()
#print(resp.status_code)

downloadCurrentFavorites()



In [ ]:
nonstreamabletracks = checkAllPlaylists()
print(nonstreamabletracks)

In [ ]:
code, response = apicall.fetchfromLibrary(item = 'playlists')
listPlaylist = [[item['id'], item['name']] for item in response['playlists']]
display(listPlaylist)

In [ ]:
tracks_df = apicall.fetchPlaylist('mp.274088456')
display(tracks_df)

### Auswertung Favorites

In [ ]:
favorites_df = pd.read_csv('favorites_2019-12-21.csv')

# replace additions like "(2009 Remix)" or "(Remastered 2016)"
# title all lowercase for better matching
favorites_df['title'] = favorites_df['title'].replace(' \([A-z,0-9 ]*\)', value='', regex=True)
favorites_df['title'] = favorites_df['title'].replace('[\'\?\’]', value='', regex=True)
favorites_df['title'] = favorites_df['title'].str.lower()

# fetch all tracks which are not streamble 
unst_df = favorites_df[favorites_df['streamable'] == False]
unst_df = unst_df.drop('streamable', 1)
#print(unst_df)

# fetch all substitutes aka double entries 
favorites_substitute_df = favorites_df[favorites_df.duplicated(subset = ['artist', 'title'], keep = 'last')]
favorites_substitute_df = favorites_substitute_df[favorites_substitute_df['streamable'] == True]
#print(favorites_substitute_df)
#print(favorites_filtered_df[favorites_filtered_df['streamable'] == False])

# cross check for a list of unstreamable track which were not replaced
unst_df = unst_df.assign(Replaced=unst_df.title.isin(favorites_substitute_df.title))
print(unst_df[unst_df['Replaced'] == False])